In [ ]:
!pip install optuna

In [ ]:
!pip install scikit-learn -q

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
files = []
classes = ['kiss', 'handshake', 'hug', 'highfive']
for class_n, c in enumerate(classes):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    
    files.extend(filtered)

In [ ]:
def convertClass(classname):
    if classname == None:
        raise Error('No classname!')
        
    mapping = {key: value for (value, key) in enumerate(classes)}
    
    return mapping[classname]

def onehot(y):
    onehot = np.zeros((1, 4))
    onehot[np.arange(1),y] = 1
    return onehot

In [ ]:
test_size = 0.2

shuffled_idx = np.random.permutation(len(files))
split_at = int(len(shuffled_idx) * (1-test_size))

train_size = len(shuffled_idx[:split_at])
test_size = len(shuffled_idx[split_at:])

In [ ]:
epochs = 20

def batcherize(files, train=True):
    if train:
        indices = shuffled_idx[:split_at]
    else:
        indices = shuffled_idx[split_at:]
        
    for _ in range(epochs+1):
        for idx in indices:
            df = np.load(files[idx])/256
            class_name = files[idx].split('/')[-2]
            class_int = convertClass(class_name)
            class_onehot = onehot(class_int)

            yield np.array([df]), class_onehot
            

In [ ]:
model = Sequential()

model.add(InputLayer((None, 128, 128,3)))

model.add(TimeDistributed(
    Conv2D(
        filters=8, 
        kernel_size=4, 
        input_shape=(128, 128, 3), 
        data_format='channels_last', 
        padding="same",
        kernel_initializer='he_normal',
    ), 
    name='Conv2D'))
#model.add(TimeDistributed(MaxPooling2D(pool_size=2), name='MaxPooling2D'))

#model.add(TimeDistributed(Flatten(), name='Flatten'))


model.add(TimeDistributed(MaxPooling2D(pool_size=4), name='MaxPooling2D'))
model.add(TimeDistributed(Flatten(), name='Flatten'))

#model.add(Dropout(0.2))

model.add(GRU(64))
model.add(Dropout(0.3))

model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.L2(0.05)))
model.add(Dropout(0.25))

#model.add(Dense(32))
#model.add(Dropout(0.2))

model.add(Dense(4, activation='softmax'))

model.summary()

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta=0.02)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

batcherize_train = batcherize(train_X, train_y)
batcherize_test = batcherize(test_X, test_y)

model.fit(
    batcherize(files, True),
    batch_size=1, 
    epochs=epochs, 
    steps_per_epoch=train_size,
    validation_data=batcherize(files, False),
    validation_steps=test_size,
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,4))

ax[0].plot(h.history['loss'], label='Training')
ax[0].plot(h.history['val_loss'], label='Test')
ax[0].set_title('Loss')
ax[0].legend()

ax[1].plot(h.history['accuracy'], label='Training')
ax[1].plot(h.history['val_accuracy'], label='Test')
ax[1].set_title('Accuracy')
ax[1].legend()